In [13]:
import cv2
import numpy as np
import imutils

class ShapeDetector:
    def __init__(self):
        pass

    def detect(self, c):
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)

        if len(approx) == 3:
            shape = "triangle"
        elif len(approx) == 4:
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
        elif len(approx) == 5:
            shape = "pentagon"
        else:
            shape = "circle"

        return shape

image_path = "img/Retail_Sales_Distribution_Network_Dashboard_With_Forecasting_3 (1).jpg"
img = cv2.imread(image_path)
kernel = np.ones((5, 5), np.uint8)
erosion = cv2.erode(img, kernel, iterations=7)
cv2.imshow("erode", erosion)
cv2.waitKey(0)
dilate = cv2.dilate(erosion, kernel, iterations=7)
cv2.imshow("dilate", dilate)
cv2.waitKey(0)
image = dilate
resized = imutils.resize(image, width=300)
cv2.imshow("resized", resized)
cv2.waitKey(0)
ratio = image.shape[0] / float(resized.shape[0])

gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
cv2.imshow("gray", gray)
cv2.waitKey(0)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imshow("blurred", blurred)
cv2.waitKey(0)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]
cv2.imshow("thresh", thresh)
cv2.waitKey(0)
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
sd = ShapeDetector()

for c in cnts:
    # Validate that contour has at least one point and correct dtype
    if len(c) < 3 or c.dtype != np.int32:
        c = np.array(c, dtype=np.int32)  # Ensure dtype is int32
    
    # Skip any contours with insufficient points or zero area
    if len(c) < 3 or cv2.contourArea(c) == 0:
        continue

    M = cv2.moments(c)

    # Ensure M["m00"] is not zero to avoid division by zero
    if M["m00"] != 0:
        cX = int((M["m10"] / M["m00"]) * ratio)
        cY = int((M["m01"] / M["m00"]) * ratio)
    else:
        cX, cY = 0, 0

    shape = sd.detect(c)

    # Multiply contour coordinates by the resize ratio, then draw
    c = c.astype("float")
    c *= ratio
    c = c.astype("int")
    cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
    cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

cv2.imshow("end", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import imutils

class ShapeDetector:
    def __init__(self):
        pass

    def detect(self, c):
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)

        if len(approx) == 3:
            shape = "triangle"
        elif len(approx) == 4:
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
        elif len(approx) == 5:
            shape = "pentagon"
        else:
            shape = "circle"

        return shape

# Load the image
image_path = "img/Predictive_Modeling_Model_Performance_1.jpg"
img = cv2.imread(image_path)

# Apply erosion and display
kernel = np.ones((5, 5), np.uint8)
erosion = cv2.erode(img, kernel, iterations=7)
cv2.imshow("Eroded", erosion)
cv2.waitKey(0)

# Apply dilation and display
dilate = cv2.dilate(erosion, kernel, iterations=7)
cv2.imshow("Dilated", dilate)
cv2.waitKey(0)

# Resize the image and display
image = dilate
resized = imutils.resize(image, width=300)
cv2.imshow("Resized", resized)
cv2.waitKey(0)
ratio = image.shape[0] / float(resized.shape[0])

# Convert to grayscale and display
gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
cv2.imshow("Gray", gray)
cv2.waitKey(0)

# Apply Gaussian blur and display
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imshow("Blurred", blurred)
cv2.waitKey(0)

# Thresholding and display
# Try adjusting the threshold value (e.g., 60)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]
cv2.imshow("Thresholded", thresh)
cv2.waitKey(0)

# Find contours using the RETR_LIST mode to detect all contours (including inner ones)
cnts = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
print(f"Number of contours detected: {len(cnts)}")

# Iterate through contours and print information
sd = ShapeDetector()

# Draw contours on the image
contour_image = resized.copy()  # Make a copy of the resized image to draw contours

for idx, c in enumerate(cnts):
    # Ensure the contour has valid points and correct dtype
    if len(c) < 3 or c.dtype != np.int32:
        c = np.array(c, dtype=np.int32)

    # Skip small contours or zero-area shapes
    if len(c) < 3 or cv2.contourArea(c) == 0:
        continue

    # Print contour index and its area
    print(f"Contour {idx+1}: Area = {cv2.contourArea(c)}")

    # Draw the contour on the image
    cv2.drawContours(contour_image, [c], -1, (0, 255, 0), 2)

    # Get the bounding box of the contour
    x, y, w, h = cv2.boundingRect(c)

    # Draw bounding box on the contour image
    cv2.rectangle(contour_image, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Calculate shape center
    M = cv2.moments(c)
    if M["m00"] != 0:
        cX = int((M["m10"] / M["m00"]) * ratio)
        cY = int((M["m01"] / M["m00"]) * ratio)
    else:
        cX, cY = 0, 0

    # Detect shape
    shape = sd.detect(c)

    # Label the shape on the image
    cv2.putText(contour_image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# Display the image with contours drawn
cv2.imshow("Contours Drawn", contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Number of contours detected: 1


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:315: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::contourArea'


: 